This is to test lane model on sim situation in difficult conditions
First we need to generate images 
We need to use python env with Carla 9.13 to avoid the bug for Windows images in 9.14 

In [15]:
#all imports
import carla #the sim library itself
import random #to pick random spawn point
import cv2 #to work with images from cameras
import numpy as np #in this example to change image representation - re-shaping
import time

In [5]:
# connect to the sim 
client = carla.Client('localhost', 2000)

In [ ]:
# load a map with challenging road without lane marking
#client.set_timeout(15)
client.load_world('Town04')

In [ ]:
# move in the sim window to find a road with just kurbs

In [6]:
world = client.get_world()

In [7]:
spectator = world.get_spectator()

In [8]:
print(spectator.get_transform())

Transform(Location(x=162.145157, y=-312.076935, z=5.093169), Rotation(pitch=-20.583956, yaw=169.282288, roll=0.000074))


In [10]:
# move to the start of that street
print(spectator.get_transform())

Transform(Location(x=195.500870, y=-311.011993, z=1.500359), Rotation(pitch=-1.347439, yaw=174.997467, roll=0.000115))


In [26]:
traffic_manager = client.get_trafficmanager(8000)

blueprints = world.get_blueprint_library().filter('*vehicle*')

spawn_points = world.get_map().get_spawn_points()
vehicle_bp = world.get_blueprint_library().filter('*model3*')

# clean up
for actor in world.get_actors().filter('*vehicle*'):
    actor.destroy()
for sensor in world.get_actors().filter('*sensor*'):
    sensor.destroy()

# ensure sync mode on 
settings = world.get_settings()
settings.synchronous_mode = True
settings.fixed_delta_seconds = 0.1
settings.no_rendering_mode = False
traffic_manager.set_synchronous_mode(True)
traffic_manager.global_percentage_speed_difference(70)

world.apply_settings(settings)
    
vehicle = world.try_spawn_actor(vehicle_bp[0], random.choice(spawn_points))
# generate traffic limited number
counter = 0
traffic = []
random.shuffle(spawn_points)
for n, transform in enumerate(spawn_points):
    counter +=1
    if counter > len(spawn_points)/3: # good volume of traffic - up to 1/3 of available spawn points
        break
    blueprint = random.choice(blueprints)
    traffic_vehicle = world.try_spawn_actor(blueprint, transform)
    if traffic_vehicle != None:# spawn the cars and set their autopilot and light state all together
        traffic.append(traffic_vehicle)
        traffic_vehicle.set_light_state(carla.VehicleLightState.LowBeam)
        traffic_vehicle.set_autopilot(True)
def sem_callback(image,data_dict):
    ########## IMPORTANT CHANGE for Semantic camera ##############
    image.convert(carla.ColorConverter.CityScapesPalette)
    data_dict['sem_image'] = np.reshape(np.copy(image.raw_data),(image.height,image.width,4))
    
def rgb_callback(image,data_dict):
    data_dict['rgb_image'] = np.reshape(np.copy(image.raw_data),(image.height,image.width,4))

if vehicle == None:
    print("Re-start the sim")
else:
    #lights always on
    vehicle.set_transform(carla.Transform(carla.Location(x=195.500870, y=-311.011993, z=1.500359),carla.Rotation(yaw=180)))
    vehicle.set_light_state(carla.VehicleLightState.LowBeam)
    vehicle.set_autopilot(True)
    
    #camera mount offset on the car - you can tweak these to each car to avoid any parts of the car being in the view
    CAMERA_POS_Z = 1.3 
    CAMERA_POS_X = 1.4 

    #normal rgb camera
    camera_bp = world.get_blueprint_library().find('sensor.camera.rgb')
    camera_bp.set_attribute('image_size_x', '640') # this ratio works in CARLA 9.13 on Windows
    camera_bp.set_attribute('image_size_y', '480')
    camera_bp.set_attribute('fov', '90')
    camera_init_trans = carla.Transform(carla.Location(z=CAMERA_POS_Z,x=CAMERA_POS_X))
    camera_rgb = world.spawn_actor(camera_bp,camera_init_trans,attach_to=vehicle)


    image_w = 640
    image_h = 480

    camera_data = {'rgb_image': np.zeros((image_h,image_w,4))}

    # this actually opens a live stream from the cameras
    camera_rgb.listen(lambda image: rgb_callback(image,camera_data))
    # get all drivable locations on the map
    all_roads = world.get_map().get_topology()
    #loop 1 time of day dry
    weather = carla.WeatherParameters(
            cloudiness=99,
            precipitation=80,
            sun_altitude_angle=10,
            precipitation_deposits =100,
            fog_density =0.9,
            wetness = 75)
    world.set_weather(weather)
    img_counter = 0
    prev_position = vehicle.get_transform()
    while img_counter < 200:
        world.tick()
        current_position = vehicle.get_transform()
        rgb_im = camera_data['rgb_image']
       
        cv2.imshow('2 cameras', rgb_im)
        if cv2.waitKey(1) == ord('q'):
            break
        #write images
        if current_position.location.distance(prev_position.location)>1:
            prev_position = current_position
            img_counter += 1
            time_grab = time.time_ns()
            cv2.imwrite('C:/SelfDrive/Semantic segmentation/test/%06d.png' % time_grab, rgb_im)
            
    cv2.destroyAllWindows()


    cv2.destroyAllWindows()
    camera_rgb.stop() 
    for actor in world.get_actors().filter('*vehicle*'):
        actor.destroy()
    for sensor in world.get_actors().filter('*sensor*'):
        sensor.destroy()

Now we have generated images in 'C:/SelfDrive/Semantic segmentation/test/

Now we need to switch to tf2 environment

In [5]:
# challenging case

import tensorflow as tf
from tensorflow.keras.models import load_model
import time
import cv2
import random
import os
import numpy as np
from matplotlib import pyplot as plt #to show images in this notebook

NAME = "UNET_LANES_WCE_LOSS-1680304258"
model = load_model("model/{}".format(NAME),compile=False)


rgb_path = 'C:/SelfDrive/Semantic segmentation/test/'
rgb_images = os.listdir(rgb_path)

for pth in rgb_images: 
    rgb_img = cv2.imread(rgb_path+pth,cv2.IMREAD_COLOR)
    img = np.float32(rgb_img /255)
    input_for_model = img[ :, :, None] 
    input_for_model = np.expand_dims(img, axis=0)
    mask = model(input_for_model,training=False)
    final_mask = (np.squeeze(mask[0].numpy()) * 255)
    gray = cv2.cvtColor(rgb_img, cv2.COLOR_BGR2GRAY)
    result = cv2.bitwise_and(gray,gray,mask = final_mask.astype(np.uint8))
    cv2.imshow('predicted mask',result)
    # Press Q on keyboard to  exit
    if cv2.waitKey(250) & 0xFF == ord('q'):
        break
# Closes all the frames
cv2.destroyAllWindows()

In [8]:
# all original images

import tensorflow as tf
from tensorflow.keras.models import load_model
import time
import cv2
import random
import os
import numpy as np
from matplotlib import pyplot as plt #to show images in this notebook

NAME = "UNET_LANES_WCE_LOSS-1680304258"
model = load_model("model/{}".format(NAME),compile=False)


rgb_path = 'C:/SelfDrive/out_sem/rgb/'
rgb_images = os.listdir(rgb_path)

for pth in rgb_images: 
    rgb_img = cv2.imread(rgb_path+pth,cv2.IMREAD_COLOR)
    img = np.float32(rgb_img /255)
    input_for_model = img[ :, :, None] 
    input_for_model = np.expand_dims(img, axis=0)
    mask = model(input_for_model,training=False)
    final_mask = (np.squeeze(mask[0].numpy()) * 255)
    gray = cv2.cvtColor(rgb_img, cv2.COLOR_BGR2GRAY)
    result = cv2.bitwise_and(gray,gray,mask = final_mask.astype(np.uint8))
    cv2.imshow('predicted mask',result)
    # Press Q on keyboard to  exit
    if cv2.waitKey(250) & 0xFF == ord('q'):
        break
# Closes all the frames
cv2.destroyAllWindows()